In [2]:
import numpy as np

import wf_psf as wf

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick
import seaborn as sns


import sys
import numpy as np
import time
import tensorflow as tf
import tensorflow_addons as tfa


In [3]:
def define_plot_style():
    # Define plot paramters
    # Use seaborn style
    sns.set()
    
    plot_style = {
        'figure.figsize': (12,8),
        'figure.dpi': 200,
        'figure.autolayout':True,
        'lines.linewidth': 2,
        'lines.linestyle': '-',
        'lines.marker': 'o',
        'lines.markersize': 10,
        'legend.fontsize': 20,
        'legend.loc': 'best',
        'axes.titlesize': 24,
        'font.size': 22
    }
    mpl.rcParams.update(plot_style)

saving_folder ='../figures/'

define_plot_style()
sns.set(font_scale=1.5)


In [4]:
dataset_base_path = '/gpfswork/rech/ynx/ulx23va/repo/wf-SEDs/WFE_sampling_test/multires_dataset/4096/'

data_4096 = np.load(dataset_base_path+'test_Euclid_res_id_005_wfeRes_4096.npy', allow_pickle=True)[()]
data_256 = np.load(dataset_base_path+'test_Euclid_res_id_005_wfeRes_256.npy', allow_pickle=True)[()]


data_4096.keys()


dict_keys(['stars', 'super_res_stars', 'positions', 'SEDs', 'zernike_coef', 'C_poly', 'parameters'])

In [6]:

train_data_4096 = np.load(dataset_base_path+'train_Euclid_res_2000_TrainStars_id_005_wfeRes_4096.npy', allow_pickle=True)[()]
train_data_256 = np.load(dataset_base_path+'train_Euclid_res_2000_TrainStars_id_005_wfeRes_256.npy', allow_pickle=True)[()]

train_data_4096.keys()


dict_keys(['stars', 'noisy_stars', 'super_res_stars', 'positions', 'SEDs', 'zernike_coef', 'C_poly', 'parameters'])

In [8]:
def compute_stats(GT_preds, preds):
    # Calculate residuals
    residuals = np.sqrt(np.mean((GT_preds - preds)**2, axis=(1, 2)))
    GT_star_mean = np.sqrt(np.mean((GT_preds)**2, axis=(1, 2)))

    # RMSE calculations
    rmse = np.mean(residuals)
    rel_rmse = 100. * np.mean(residuals / GT_star_mean)

    # STD calculations
    std_rmse = np.std(residuals)
    std_rel_rmse = 100. * np.std(residuals / GT_star_mean)

    # Print RMSE values
    print('Absolute RMSE:\t %.4e \t +/- %.4e' % (rmse, std_rmse))
    print('Relative RMSE:\t %.4e %% \t +/- %.4e %%' % (rel_rmse, std_rel_rmse))

    return rmse, rel_rmse, std_rmse, std_rel_rmse



## Base error resolution x1

In [13]:

GT_preds = data_4096['stars']
preds = data_256['stars']

_ = compute_stats(GT_preds, preds)



Absolute RMSE:	 3.4710e-05 	 +/- 7.3129e-06
Relative RMSE:	 4.6788e-01 % 	 +/- 1.0713e-01 %


In [14]:
preds.shape

(400, 32, 32)

## Base error resolution x3

In [12]:

GT_preds = train_data_4096['super_res_stars']
preds = train_data_256['super_res_stars']

_ = compute_stats(GT_preds, preds)


GT_preds = train_data_4096['stars']
preds = train_data_256['stars']

_ = compute_stats(GT_preds, preds)


Absolute RMSE:	 8.5895e-06 	 +/- 1.3838e-06
Relative RMSE:	 5.7694e-01 % 	 +/- 1.0029e-01 %
Absolute RMSE:	 3.5417e-05 	 +/- 7.6075e-06
Relative RMSE:	 4.7907e-01 % 	 +/- 1.0847e-01 %


In [11]:

GT_preds = data_4096['super_res_stars']
preds = data_256['super_res_stars']

_ = compute_stats(GT_preds, preds)


Absolute RMSE:	 8.5248e-06 	 +/- 1.3512e-06
Relative RMSE:	 5.7081e-01 % 	 +/- 1.0100e-01 %


## Generate x3 reconstructions

In [7]:
args = {
    'model': 'poly',
    'model_eval': 'poly',
    'base_id_name': '_wfe_study_resx3_256_bis_',
    'id_name': '_wfe_study_resx3_256_bis_1',
    'train_dataset_file': 'train_Euclid_res_2000_TrainStars_id_005_wfeRes_4096.npy',
    'test_dataset_file': 'test_Euclid_res_id_005_wfeRes_4096.npy',
    'n_epochs_param': [15, 15],
    'n_epochs_non_param': [100, 50],
    'n_zernikes': 15,
    'gt_n_zernikes': 45,
    'pupil_diameter': 256,
    'oversampling_rate': 3.,
    'output_q': 3.,
    'output_dim': 32,
    'batch_size': 32,
    'd_max': 2,
    'x_lims': [0, 1e3],
    'y_lims': [0, 1e3],
    'd_max_nonparam': 5,
    'n_bins_lda': 20,
    'eval_batch_size': 16,
    'interpolation_type': 'none',
    'l_rate_param': [0.01, 0.004],
    'l_rate_non_param': [0.1, 0.06],
    'saved_model_type': 'checkpoint',
    'saved_cycle': 'cycle2',
    'total_cycles': 2,
    'use_sample_weights': True,
    'l2_param': 0.,
    'cycle_def': 'complete',
    'suffix_id_name': ['1','2','3','4','5'],
    'star_numbers': [1, 2, 3, 4, 5],
    'train_opt': True,
    'eval_opt':  True,
    'plot_opt': True,
    'base_path':  '/gpfswork/rech/ynx/ulx23va/repo/wf-SEDs/model_WFE_size/wf-outputs/',
    'dataset_folder':  '/gpfswork/rech/ynx/ulx23va/repo/wf-SEDs/WFE_sampling_test/multires_dataset/4096/',
    'metric_base_path':  '/gpfswork/rech/ynx/ulx23va/repo/wf-SEDs/model_WFE_size/wf-outputs/metrics/wfe_study_resx3_bis/',
    'chkp_save_path':  '/gpfswork/rech/ynx/ulx23va/repo/wf-SEDs/model_WFE_size/wf-outputs/chkp/wfe_study_resx3_bis/',
    'log_folder':  'log-files/wfe_study_resx3_bis/',
    'model_folder':  'chkp/wfe_study_resx3_bis/',
    'optim_hist_folder':  'optim-hist/wfe_study_resx3_bis/',
    'plots_folder':  'plots/wfe_study_resx3_bis/',
}


In [8]:
# Load models
test_dataset = np.load(args['dataset_folder'] + args['test_dataset_file'], allow_pickle=True)[()]
test_stars = test_dataset['stars']
test_pos = test_dataset['positions']
test_SEDs = test_dataset['SEDs']
# test_zernike_coef = test_dataset['zernike_coef']
test_C_poly = test_dataset['C_poly']
test_parameters = test_dataset['parameters']


tf_test_pos = tf.convert_to_tensor(test_dataset['positions'], dtype=tf.float32)


2022-08-22 01:06:26.374028: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-22 01:06:27.044185: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30986 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8a:00.0, compute capability: 7.0


In [9]:

# Iterate through the model (bis_id), then through the pupil_diameter
dataset_list = [
    '064',
    '128',
    '256',
    '256_benchmark',
]

pupil_diameter_list = [64, 128, 256, 256]

n_realisations = 15



In [10]:

for it_j in range(len(pupil_diameter_list)):
    
    
    pupil_diameter = pupil_diameter_list[it_j]
    dataset_name = dataset_list[it_j]
    result_x3_list = []
    
    for idx in range(1, n_realisations+1):

        run_id_name = 'poly_wfe_study_resx3_' + dataset_name + '_bis_' + str(idx)
        print(run_id_name)

        n_bins_lda=args['n_bins_lda']
        output_Q=1
        output_dim=64
        batch_size=args['eval_batch_size']

        # Define weights
        weights_paths = args['chkp_save_path'] + 'chkp_callback_' + run_id_name + '_' + args['saved_cycle']


        ## Prepare models
        # Generate Zernike maps
        zernikes = wf.utils.zernike_generator(
            n_zernikes=args['n_zernikes'], wfe_dim=pupil_diameter
        )
        # Now as cubes
        np_zernike_cube = np.zeros((len(zernikes), zernikes[0].shape[0], zernikes[0].shape[1]))

        for it in range(len(zernikes)):
            np_zernike_cube[it, :, :] = zernikes[it]

        np_zernike_cube[np.isnan(np_zernike_cube)] = 0
        tf_zernike_cube = tf.convert_to_tensor(np_zernike_cube, dtype=tf.float32)

        # Prepare np input
        simPSF_np = wf.SimPSFToolkit(
            zernikes,
            max_order=args['n_zernikes'],
            pupil_diameter=pupil_diameter,
            output_dim=args['output_dim'],
            oversampling_rate=args['oversampling_rate'],
            output_Q=args['output_q']
        )
        simPSF_np.gen_random_Z_coeffs(max_order=args['n_zernikes'])
        z_coeffs = simPSF_np.normalize_zernikes(simPSF_np.get_z_coeffs(), simPSF_np.max_wfe_rms)
        simPSF_np.set_z_coeffs(z_coeffs)
        simPSF_np.generate_mono_PSF(lambda_obs=0.7, regen_sample=False)

        # Obscurations
        obscurations = simPSF_np.generate_pupil_obscurations(N_pix=pupil_diameter, N_filter=2)
        tf_obscurations = tf.convert_to_tensor(obscurations, dtype=tf.complex64)


        # Initialize the model
        tf_semiparam_field = wf.tf_psf_field.TF_SemiParam_field(
            zernike_maps=tf_zernike_cube,
            obscurations=tf_obscurations,
            batch_size=args['batch_size'],
            output_Q=args['output_q'],
            d_max_nonparam=args['d_max_nonparam'],
            l2_param=args['l2_param'],
            output_dim=args['output_dim'],
            n_zernikes=args['n_zernikes'],
            d_max=args['d_max'],
            x_lims=args['x_lims'],
            y_lims=args['y_lims']
        )

        tf_semiparam_field.load_weights(weights_paths)



        # Save original output_Q and output_dim
        original_out_Q = tf_semiparam_field.output_Q
        original_out_dim = tf_semiparam_field.output_dim

        # Set the required output_Q and output_dim parameters in the models
        tf_semiparam_field.set_output_Q(output_Q=output_Q, output_dim=output_dim)


        # Need to compile the models again
        tf_semiparam_field = wf.tf_psf_field.build_PSF_model(tf_semiparam_field)


        # Generate SED data list
        packed_SED_data = [
            wf.utils.generate_packed_elems(_sed, simPSF_np, n_bins=n_bins_lda) for _sed in test_SEDs
        ]


        # Prepare inputs
        tf_packed_SED_data = tf.convert_to_tensor(packed_SED_data, dtype=tf.float32)
        tf_packed_SED_data = tf.transpose(tf_packed_SED_data, perm=[0, 2, 1])
        pred_inputs = [tf_test_pos, tf_packed_SED_data]

        # PSF model
        predictions = tf_semiparam_field.predict(x=pred_inputs, batch_size=16)

        # Save results
        result_x3_list.append(predictions)

    # Save the realisations
    np.save(
        saving_folder+'model_x3_res_'+dataset_name+'.npy',
        np.array(result_x3_list),
        allow_pickle=True
    )




poly_wfe_study_resx3_064_bis_1
poly_wfe_study_resx3_064_bis_2
poly_wfe_study_resx3_064_bis_3
poly_wfe_study_resx3_064_bis_4
poly_wfe_study_resx3_064_bis_5
poly_wfe_study_resx3_064_bis_6
poly_wfe_study_resx3_064_bis_7
poly_wfe_study_resx3_064_bis_8


poly_wfe_study_resx3_064_bis_9
poly_wfe_study_resx3_064_bis_10
poly_wfe_study_resx3_064_bis_11
poly_wfe_study_resx3_064_bis_12
poly_wfe_study_resx3_064_bis_13
poly_wfe_study_resx3_064_bis_14
poly_wfe_study_resx3_064_bis_15
poly_wfe_study_resx3_128_bis_1
poly_wfe_study_resx3_128_bis_2
poly_wfe_study_resx3_128_bis_3
poly_wfe_study_resx3_128_bis_4
poly_wfe_study_resx3_128_bis_5


poly_wfe_study_resx3_128_bis_6
poly_wfe_study_resx3_128_bis_7
poly_wfe_study_resx3_128_bis_8


poly_wfe_study_resx3_128_bis_9
poly_wfe_study_resx3_128_bis_10
poly_wfe_study_resx3_128_bis_11
poly_wfe_study_resx3_128_bis_12
poly_wfe_study_resx3_128_bis_13
poly_wfe_study_resx3_128_bis_14
poly_wfe_study_resx3_128_bis_15
poly_wfe_study_resx3_256_bis_1
poly_wfe_study_resx3_256_bis_2


poly_wfe_study_resx3_256_bis_3
poly_wfe_study_resx3_256_bis_4
poly_wfe_study_resx3_256_bis_5
poly_wfe_study_resx3_256_bis_6
poly_wfe_study_resx3_256_bis_7
poly_wfe_study_resx3_256_bis_8
poly_wfe_study_resx3_256_bis_9
poly_wfe_study_resx3_256_bis_10
poly_wfe_study_resx3_256_bis_11
poly_wfe_study_resx3_256_bis_12
poly_wfe_study_resx3_256_bis_13
poly_wfe_study_resx3_256_bis_14


poly_wfe_study_resx3_256_bis_15
poly_wfe_study_resx3_256_benchmark_bis_1
poly_wfe_study_resx3_256_benchmark_bis_2
poly_wfe_study_resx3_256_benchmark_bis_3
poly_wfe_study_resx3_256_benchmark_bis_4
poly_wfe_study_resx3_256_benchmark_bis_5
poly_wfe_study_resx3_256_benchmark_bis_6
poly_wfe_study_resx3_256_benchmark_bis_7
poly_wfe_study_resx3_256_benchmark_bis_8
poly_wfe_study_resx3_256_benchmark_bis_9
poly_wfe_study_resx3_256_benchmark_bis_10
poly_wfe_study_resx3_256_benchmark_bis_11


poly_wfe_study_resx3_256_benchmark_bis_12
poly_wfe_study_resx3_256_benchmark_bis_13
poly_wfe_study_resx3_256_benchmark_bis_14
poly_wfe_study_resx3_256_benchmark_bis_15


# Evaluate the results

In [11]:
GT_preds = data_4096['super_res_stars']
preds = data_256['super_res_stars']

_ = compute_stats(GT_preds, preds)


Absolute RMSE:	 8.5248e-06 	 +/- 1.3512e-06
Relative RMSE:	 5.7081e-01 % 	 +/- 1.0100e-01 %


In [12]:


dataset_list = [
    '064',
    '128',
    '256',
    '256_benchmark',
]

GT_preds = data_4096['super_res_stars']

save_dict = {}

# rmse, rel_rmse, std_rmse, std_rel_rmse

for it_j in range(len(dataset_list)):
    rel_rmse_list = []  
    std_rel_rmse_list = [] 
    
    model = np.load(saving_folder+'model_x3_res_'+dataset_list[it_j]+'.npy', allow_pickle=True)
    
    for it in range(model.shape[0]):

        preds = model[it,:,:,:]

        rmse, rel_rmse, std_rmse, std_rel_rmse = compute_stats(GT_preds, preds)

        rel_rmse_list.append(rel_rmse)
        std_rel_rmse_list.append(std_rel_rmse)
    
    save_dict['model_'+dataset_list[it_j]+'_rel_rmse'] = np.array(rel_rmse_list)
    save_dict['model_'+dataset_list[it_j]+'_std_rel_rmse'] = np.array(std_rel_rmse_list)


Absolute RMSE:	 9.4269e-05 	 +/- 2.5365e-05
Relative RMSE:	 6.4396e+00 % 	 +/- 2.4688e+00 %
Absolute RMSE:	 6.6703e-05 	 +/- 2.6952e-05
Relative RMSE:	 4.5149e+00 % 	 +/- 2.0408e+00 %
Absolute RMSE:	 1.0155e-04 	 +/- 4.6715e-05
Relative RMSE:	 7.0313e+00 % 	 +/- 4.1970e+00 %
Absolute RMSE:	 7.6872e-05 	 +/- 2.4688e-05
Relative RMSE:	 5.1850e+00 % 	 +/- 1.8659e+00 %
Absolute RMSE:	 8.3664e-05 	 +/- 1.9890e-05
Relative RMSE:	 5.6681e+00 % 	 +/- 1.6947e+00 %
Absolute RMSE:	 8.4165e-05 	 +/- 2.1581e-05
Relative RMSE:	 5.7144e+00 % 	 +/- 1.9407e+00 %
Absolute RMSE:	 8.3305e-05 	 +/- 1.8980e-05
Relative RMSE:	 5.6576e+00 % 	 +/- 1.7825e+00 %
Absolute RMSE:	 9.4091e-05 	 +/- 3.2872e-05
Relative RMSE:	 6.4697e+00 % 	 +/- 3.0367e+00 %
Absolute RMSE:	 9.0510e-05 	 +/- 2.6192e-05
Relative RMSE:	 6.1777e+00 % 	 +/- 2.3060e+00 %
Absolute RMSE:	 6.4772e-05 	 +/- 2.5626e-05
Relative RMSE:	 4.3889e+00 % 	 +/- 1.9543e+00 %
Absolute RMSE:	 8.2584e-05 	 +/- 2.7403e-05
Relative RMSE:	 5.6444e+00 % 	 +/- 2

In [13]:

np.save(saving_folder+'result_dict_total_reps.npy', save_dict, allow_pickle=True)


In [14]:
save_dict

{'model_064_rel_rmse': array([6.43958849, 4.5148942 , 7.03129339, 5.18500449, 5.66809588,
        5.71443545, 5.65758543, 6.46967079, 6.17768274, 4.38893771,
        5.64442946, 5.58807755, 6.81902082, 4.94931877, 6.60063669]),
 'model_064_std_rel_rmse': array([2.46884952, 2.04078886, 4.19700985, 1.8659101 , 1.69467889,
        1.94065863, 1.78249998, 3.03667582, 2.30599106, 1.95426579,
        2.37413141, 2.36298788, 2.91554769, 2.1565339 , 2.8102065 ]),
 'model_128_rel_rmse': array([5.42097046, 2.11293908, 2.10626005, 3.45872813, 3.28045238,
        3.67064308, 3.07061397, 4.19846729, 4.5723978 , 2.76683228,
        4.72089701, 5.16129057, 5.65956429, 5.4985767 , 3.60274507]),
 'model_128_std_rel_rmse': array([4.0749473 , 1.29302873, 1.20299272, 1.18403576, 1.43188567,
        1.10953506, 1.49602465, 1.9786871 , 1.61180811, 1.34480574,
        2.28292611, 1.90541314, 4.05664667, 1.89366435, 1.53948893]),
 'model_256_rel_rmse': array([4.18412105, 6.33608123, 3.2794323 , 4.67587151, 4.

In [21]:
print('Last 10 repetitions:'),print('')
print('Mean: ', np.mean(save_dict['model_064_rel_rmse'][5:]))
print('Median: ', np.median(save_dict['model_064_rel_rmse'][5:]))
print('Std dev reps: ', np.std(save_dict['model_064_rel_rmse'][5:]))
print('Mean Std dev: ', np.mean(save_dict['model_064_std_rel_rmse'][5:]))
print('')
print('Mean: ', np.mean(save_dict['model_128_rel_rmse'][5:]))
print('Median: ', np.median(save_dict['model_128_rel_rmse'][5:]))
print('Std dev reps: ', np.std(save_dict['model_128_rel_rmse'][5:]))
print('Mean Std dev: ', np.mean(save_dict['model_128_std_rel_rmse'][5:]))
print('')
print('Mean: ', np.mean(save_dict['model_256_rel_rmse'][5:]))
print('Median: ', np.median(save_dict['model_256_rel_rmse'][5:]))
print('Std dev reps: ', np.std(save_dict['model_256_rel_rmse'][5:]))
print('Mean Std dev: ', np.mean(save_dict['model_256_std_rel_rmse'][5:]))
print('')
print('Mean: ', np.mean(save_dict['model_256_benchmark_rel_rmse'][5:]))
print('Median: ', np.median(save_dict['model_256_benchmark_rel_rmse'][5:]))
print('Std dev reps: ', np.std(save_dict['model_256_benchmark_rel_rmse'][5:]))
print('Mean Std dev: ', np.mean(save_dict['model_256_benchmark_std_rel_rmse'][5:]))
print('')


Last 10 repetitions:

Mean:  5.800979539380679
Median:  5.686010443418695
Std dev reps:  0.7128806375529859
Mean Std dev:  2.3639498659359495

Mean:  4.292202804962117
Median:  4.385432541559398
Std dev reps:  0.9503749912314566
Mean Std dev:  1.9218999862259687

Mean:  4.466562247264386
Median:  4.339361947000835
Std dev reps:  0.7505609789278637
Mean Std dev:  2.021885716077639

Mean:  4.936639088891042
Median:  4.538246798904764
Std dev reps:  1.0475086011399142
Mean Std dev:  2.493699321761354



In [22]:
print('All repetitions:'),print('')
print('Mean: ', np.mean(save_dict['model_064_rel_rmse']))
print('Median: ', np.median(save_dict['model_064_rel_rmse']))
print('Std dev reps: ', np.std(save_dict['model_064_rel_rmse']))
print('Mean Std dev: ', np.mean(save_dict['model_064_std_rel_rmse']))
print('')
print('Mean: ', np.mean(save_dict['model_128_rel_rmse']))
print('Median: ', np.median(save_dict['model_128_rel_rmse']))
print('Std dev reps: ', np.std(save_dict['model_128_rel_rmse']))
print('Mean Std dev: ', np.mean(save_dict['model_128_std_rel_rmse']))
print('')
print('Mean: ', np.mean(save_dict['model_256_rel_rmse']))
print('Median: ', np.median(save_dict['model_256_rel_rmse']))
print('Std dev reps: ', np.std(save_dict['model_256_rel_rmse']))
print('Mean Std dev: ', np.mean(save_dict['model_256_std_rel_rmse']))
print('')
print('Mean: ', np.mean(save_dict['model_256_benchmark_rel_rmse']))
print('Median: ', np.median(save_dict['model_256_benchmark_rel_rmse']))
print('Std dev reps: ', np.std(save_dict['model_256_benchmark_rel_rmse']))
print('Mean Std dev: ', np.mean(save_dict['model_256_benchmark_std_rel_rmse']))
print('')


All repetitions:

Mean:  5.789911456087106
Median:  5.66809587501981
Std dev reps:  0.7768630456685823
Mean Std dev:  2.3937823920223473

Mean:  3.953425209665083
Median:  3.670643078933697
Std dev reps:  1.1497983546956478
Mean Std dev:  1.8937260029447216

Mean:  4.529558347232991
Median:  4.628339828397148
Std dev reps:  0.8451641405007336
Mean Std dev:  2.1011167817754863

Mean:  4.513860014111088
Median:  4.525749019511434
Std dev reps:  1.2218316982031194
Mean Std dev:  2.2445340530898306



In [17]:

old_save_dict = np.load(saving_folder+'result_dict.npy', allow_pickle=True)[()]


In [18]:
old_save_dict

{'model_064_rel_rmse': array([6.43958849, 4.5148942 , 7.03129339, 5.18500449, 5.66809588]),
 'model_064_std_rel_rmse': array([2.46884952, 2.04078886, 4.19700985, 1.8659101 , 1.69467889]),
 'model_128_rel_rmse': array([5.42097046, 2.11293908, 2.10626005, 3.45872813, 3.28045238]),
 'model_128_std_rel_rmse': array([4.0749473 , 1.29302873, 1.20299272, 1.18403576, 1.43188567]),
 'model_256_rel_rmse': array([4.18412105, 6.33608123, 3.2794323 , 4.67587151, 4.80224664]),
 'model_256_std_rel_rmse': array([1.85443559, 3.54970007, 1.32788761, 2.23428099, 2.3315903 ]),
 'model_256_benchmark_rel_rmse': array([2.58978528, 3.92488443, 5.63453092, 3.48876644, 2.70354226]),
 'model_256_benchmark_std_rel_rmse': array([1.40024367, 1.20963646, 3.8965619 , 1.1530999 , 1.07147565])}

In [19]:

print('Mean: ', np.mean(old_save_dict['model_256_benchmark_rel_rmse']))
print('Median: ', np.median(old_save_dict['model_256_benchmark_rel_rmse']))
print('Std dev reps: ', np.std(old_save_dict['model_256_benchmark_rel_rmse']))
print('Mean Std dev: ', np.mean(old_save_dict['model_256_benchmark_std_rel_rmse']))
print('')



Mean:  3.668301864551181
Median:  3.4887664380025654
Std dev reps:  1.1007346096343087
Mean Std dev:  1.7462035157467848



In [20]:
GT_preds = data_4096['super_res_stars']
preds = data_256['super_res_stars']

_ = compute_stats(GT_preds, preds)

Absolute RMSE:	 8.5248e-06 	 +/- 1.3512e-06
Relative RMSE:	 5.7081e-01 % 	 +/- 1.0100e-01 %
